## Stack
Stack is a `LIFO`, Last In First Out data structure. It is a list with restriction that insertion and deletion occurs only at one end.  
Stack can be implemented using arrays or using linked list.

### Array based implementation

In [ ]:
class SimpleArrayStack<T> {
    private final int MAX_SIZE;
    private final Object[] elements;

    private int top = -1;

    public SimpleArrayStack(int size) {
        MAX_SIZE = size;
        elements = new Object[MAX_SIZE];
    }

    public boolean isEmpty() {
        return top == -1;
    }

    public void push(T x) {
        if (top < MAX_SIZE - 1) {
            elements[++top] = x;
        } else {
            throw new IllegalArgumentException("Stack is full");
        }
    }

    public T pop() {
        if (isEmpty()) {
            throw new IllegalArgumentException("Queue is empty");
        }

        top--;
        return (T) elements[top + 1];
    }
}

We can modify the above code to extend the array when the array completely fills up. In such case the time complexities are:
1. push : Best case $O(1)$, Worst case (when we have to extend) $O(n)$, Average case $O(1)$
2. pop : $O(1)$

In [ ]:
class ResizableArrayStack<T> {
    private int arraySize = 5;
    private Object[] elements = new Object[arraySize];

    private int top = -1;

    public boolean isEmpty() {
        return top == -1;
    }

    public void push(T x) {
        if (top == arraySize - 1) {
            resize(2 * arraySize);
        }

        elements[++top] = x;
    }

    public T pop() {
        if (isEmpty()) {
            throw new IllegalArgumentException("Stack is empty");
        } else if (arraySize > 3 * top) {
            resize(arraySize / 2);
        }

        top--;
        return (T) elements[top + 1];
    }

    private void resize(int newSize) {
        Object[] newElements = new Object[newSize];

        for (int i = 0; i < elements.length; i++) {
            newElements[i] = elements[i];
        }

        elements = newElements;
        arraySize = newSize;
    }
}

### A Note on Time Complexity of Insertion of Elements in a Resizable Array
Whenever the array is full we double the size of array, copy the elements from old array to the new one. So what is the time complexity? To calculate the time complexity of addition of element into a resizable array, we do *[amortized time complexity](https://brilliant.org/wiki/amortized-analysis/)* which is basically weighted average time complexity.
  
Suppose we have to insert $n$ elements into an array and we start with array having size $1$.  
So as soon as we add $1$ element, we create a new array with double the size and copy that $1$ element to the new array. When we add one more element, we double array and copy over $2$ elements. Adding the third element takes constant time. So for $n$ elements, the times are: 

$$1 \rightarrow 1$$
$$2 \rightarrow 2 = 2^0 + 1$$
$$3 \rightarrow 3 = 2^1 + 1$$
$$4 \rightarrow 1$$
$$5 \rightarrow 5 = 2^2 + 1$$
$$6 \rightarrow 1$$
$$\ldots$$

<img src="images/amortized_analysis.png" />

### Linked List Based Implementation

In [ ]:
class LinkedListStack<T> {
    private class Node {
        public T data;
        public Node next;

        public Node(T data) {
            this.data = data;
        }
    }

    private Node top;

    public boolean isEmpty() {
        return top == null;
    }

    public void push(T x) {
        Node temp = new Node(x);
        temp.next = top;
        top = temp;
    }

    public T pop() {
        if (isEmpty()) {
            throw new IllegalArgumentException("Stack is empty");
        }

        T value = top.data;
        top = top.next;
        return value;
    }
}

Linked List based stack implementation has the same push and pop time complexity as array based implementation (as long as array based implementation has fixed length).

### Java Implementation
We can use either of `Stack`, `ArrayDeque` or `ArrayList`.

**Stack:** a `Vector` based implementation
- Instantiate : `Stack<Integer> stack = new Stack<>();`
- Size: `stack.size();`
- Push: `stack.push(5)`
- Pop: `int item = stack.pop()`
- Iterate: `for(Integer element: stack)`

**ArrayDeque:** prefer this.
- Instantiate : `ArrayDeque<Integer> stack = new ArrayDeque<>();`
- Size: `stack.size();`
- Push: `stack.push(5)`, `stack.addFirst(5)`
- Pop: `int item = stack.pop()` or `int item = stack.removeFirst()`
- Iterate: `for(Integer element: stack)`

**ArrayList:**
- Instantiate : `ArrayList<Integer> stack = new ArrayList<>();`
- Size: `stack.size();`
- Push: `stack.add(5)`
- Pop: `int item = stack.remove(stack.size() - 1)`
- Iterate: `for(Integer element: stack)`

## Infix Postfix and Prefix
A normal arithmetic expression comprises of opertor in between operands. Expression is evaluated from left to right and we have to take into consideration the precedence of various operators. Paranthesis is used to specify what is evaluated first. The expression `A + B * C + D` can be rewritten as `((A + (B * C)) + D)` to show that the multiplication happens first, followed by the leftmost addition.  

**Prefix:** notation requires that all operators precede the two operands that they work on  
**Postfix:** notation requires that its operators come after the corresponding operands  

Examples:
```
| Infix             | Prefix        | Postfix       |
|-------------------|---------------|---------------|
| A + B * C + D     | + + A * B C D | A B C * + D + |
| (A + B) * (C + D) | * + A B + C D | A B + C D + * |
| A * B + C * D     | + * A B * C D | A B * C D * + |
```

Prefix and postfix notations have the benefit of not requiring paranthesis to determine the order of operations.

**Converting infix to postfix and prefix:** if the infix operation is completely paranthesized, we traverse the expression and a) in case of postfix replace ')' with the operator b) in case of prefix replace '(' with the operator.

![Postfix](https://runestone.academy/runestone/books/published/pythonds/_images/moveright.png)
![Prefix](https://runestone.academy/runestone/books/published/pythonds/_images/moveleft.png)  

In [1]:
public String toPostFix(String infix) {
    StringBuilder postfix = new StringBuilder();
    
    // Holds operators    
    ArrayDeque<String> stack = new ArrayDeque<>();
    
    // Considering input has space separated symbols
    String[] tokens = infix.split(" ");
    
    for(String token: tokens) {
        // if not operator, just append to output
        if(!(token.equals("+") || token.equals("-") || token.equals("*") || token.equals("/")
            || token.equals("(") || token.equals(")"))) {
            postfix.append(token);
        // add opening paranthesis to the stack
        } else if(token.equals("(")) {
            stack.push(token);
        // on encountering closing paranthesis, remove
        // everything from stack (and append) till you
        // get to corresponding opening paranthesis
        } else if(token.equals(")")) {
            String popped = null;
            while(!stack.isEmpty() && !(popped = stack.pop()).equals("(")) {
                postfix.append(popped);
            }
        // For other operators before inserting into stack
        // remove all operators which have equal or greater
        // precedence
        } else if(token.equals("+") || token.equals("-")) {
            while(!stack.isEmpty()) {
                String top = stack.peek();
                if(top.equals("+") || top.equals("-") ||
                   top.equals("*") || top.equals("/")) {
                    postfix.append(stack.pop());
                } else {
                    break;
                }
            }
            
            stack.push(token);
        } else if(token.equals("*") || token.equals("/")) {
            while(!stack.isEmpty()) {
                String top = stack.peek();
                if(top.equals("*") || top.equals("/")) {
                    postfix.append(stack.pop());
                } else {
                    break;
                }
            }
            
            stack.push(token);
        }
    }
    
    // Empty out stack
    while(!stack.isEmpty()) {
        postfix.append(stack.pop());
    }

    return new String(postfix);
}

System.out.println("Infix: A + ( B * C ) + D\tPostfix: " + toPostFix("A + ( B * C ) + D"));
System.out.println("Infix: ( A + B ) * ( C + D )\tPostfix: " + toPostFix("( A + B ) * ( C + D )"));
System.out.println("Infix: ( A * B ) + ( C * D )\tPostfix: " + toPostFix("( A * B ) + ( C * D )"));

Infix: A + ( B * C ) + D	Postfix: ABC*+D+
Infix: ( A + B ) * ( C + D )	Postfix: AB+CD+*
Infix: ( A * B ) + ( C * D )	Postfix: AB*CD*+


In [2]:
public int evaluate(String postfix) {
    // Only numbers are added to this stack
    ArrayDeque<Integer> stack = new ArrayDeque<>();

    // Considering input has space separated symbols
    String[] tokens = postfix.trim().split(" ");

    for(String token: tokens) {
        // push numbers to stack
        if(!(token.equals("+") || token.equals("-") ||
             token.equals("*") || token.equals("/"))) {
            stack.push(Integer.parseInt(token));
        } else {
            int rightOperand = stack.pop();
            int leftOperand = stack.pop();
            int result = 0;

            // evaluate binary expression as per operator
            // then push result to stack again
            if(token.equals("+")) {
                result = leftOperand + rightOperand;
            } else if(token.equals("-")) {
                result = leftOperand - rightOperand;
            } else if(token.equals("*")) {
                result = leftOperand * rightOperand;
            } else if(token.equals("/")) {
                result = leftOperand / rightOperand;
            }

            stack.push(result);
        }
    }

    return stack.pop();
}

System.out.println("Expression: 12 3 5 * + 9 +\tResult: " + evaluate("12 3 5 * + 9 +"));

Expression: 12 3 5 * + 9 +	Result: 36


**Q 1:** Implement a function `getMin()` which returns the minimum element in the stack.  
**Answer:** So get minimum element of stack, we need to make use of two stacks. One main stack and the other stack which holds current minimum at the top.

In [2]:
public class StackWithMin {
    ArrayDeque<Integer> stack = new ArrayDeque<>();
    ArrayDeque<Integer> minStack;
    
    public StackWithMin() {
        stack = new ArrayDeque<>();
        minStack = new ArrayDeque<>();
    }
    
    public void push(int x) {
        stack.push(x);
        
        if (minStack.isEmpty()) {
            minStack.push(x);
        } else if (x <= minStack.peek()) {
            minStack.push(x);
        }
    }
    
    public int pop() {
        Integer popped = stack.pop();
        
        if (popped.equals(minStack.peek())) {
            minStack.pop();
        }
        
        return popped;
    }
    
    public int getMin() {
        return minStack.peek();
    }
}

StackWithMin stack = new StackWithMin();
stack.push(7); stack.push(8); stack.push(3);
System.out.println(stack.getMin());

stack.push(1); stack.push(0); stack.push(0);
stack.pop();
System.out.println(stack.getMin());

3
0


How can we maintain minimum without using extra space? If all the numbers to be inserted into the stack are positive then we can follow the below steps. Consider the array is `6, 3, 7, 8, 2`. We insert 6 first into the stack. The variable `min_stack = 6`. Next when we push 3, `min_stack = 3`, however the value inserted into the stack will be `3 - min_stack` which is `3 - 6 = -3`. We will insert 7 and 8 as it is. When we insert 2, we make `min_stack = 2`, however we insert `2 - 3 = -1` into the stack. And so on. The stack now is `6, -3, 7, 8, -1`.  
Now when we pop elements and it is negative, then to get to the original number we need to follow the following equation: `x - prev_min_stack = top_of_stack`. But how to get the previous min from current min? `prev_min_stack = min_stack - top_of_stack`. `prev_min_stack = 2 - (-1) = 3`. So the popped elements is `x - 3 = -1 = 2`. Make `min_stack = prev_min_stack`.

**Q 2:** Sort a stack  
**Answer:**

In [4]:
public void sort(ArrayDeque<Integer> input) {
    ArrayDeque<Integer> temp = new ArrayDeque<>();

    while (!input.isEmpty()) {
        int popped = input.pop();

        // To simplify the implementation, we can just keep the last
        // else branch
        if (temp.isEmpty()) {
            temp.push(popped);
        } else if (popped >= temp.peek()) {
            temp.push(popped);
        } else {
            while (!temp.isEmpty() && popped < temp.peek()) {
                input.push(temp.pop());
            }
            temp.push(popped);
        }
    }

    while (!temp.isEmpty()) {
        input.push(temp.pop());
    }
}

ArrayDeque<Integer> s1 = new ArrayDeque<>();
s1.push(2);
s1.push(3);
s1.push(1);
s1.push(5);
s1.push(1);
sort(s1);
System.out.println(s1);

[1, 1, 2, 3, 5]


**Q 3:** Given a string, remove all the consecutive duplicate elements. For example, if the string is `abbck`, then return `ack`. However consider `kabbal`. It will convert to `kaal` and then finally to `kl`. Finally if we pass `aaa` as input, we should get `a` .  
**Answer:**

In [5]:
public String removeDuplicates(String input) {
    char[] chars = input.toCharArray();
    ArrayDeque<Character> stack = new ArrayDeque<>();

    for (char c : chars) {
        if (!stack.isEmpty() && stack.peek() == c) {
            stack.pop();
        }

        stack.push(c);
    }

    StringBuilder builder = new StringBuilder();
    while (!stack.isEmpty()) {
        builder.append(stack.pop());
    }

    return builder.reverse().toString();
}

System.out.println(removeDuplicates("abbcgeet"));
System.out.println(removeDuplicates("kabbbal"));
System.out.println(removeDuplicates("kabbal"));
System.out.println(removeDuplicates("aaa"));
System.out.println(removeDuplicates("abbck"));

abcget
kabal
kabal
a
abck


**Q 4:** Determine whether the given set of paranthesis is valid or not. Paranthesis can be either of the following: `[, {, (, ), }, ]`  
**Answer:** Add opening braces to the stack. If you get closing brace, pop and the stack and check if it matches the closing brace.

In [6]:
public boolean validParenthesis(String input) {
    if (input.isEmpty()) return true;

    ArrayDeque<Character> stack = new ArrayDeque<>();
    char[] chars = input.toCharArray();
    for (char c : chars) {
        if (c == '[' || c == '{' || c == '(') {
            stack.push(c);
        } else if (stack.isEmpty() && (c == ']' || c == '}' || c == ')')) {
            return false;
        } else if (!stack.isEmpty()) {
            char popped = stack.pop();
            if ((c == ']' && popped != '[') || (c == '}' && popped != '{') || (c == ')' && popped != '(')) {
                return false;
            }
        }
    }

    return stack.isEmpty();
}

System.out.println(validParenthesis("{[)(]}"));
System.out.println(validParenthesis("{[()]}"));

false
true


**Q 5:** Given an array, for each element find the closest number (to the right) greater than that number. For example, if the array is `[10, 7, 4, 2, 9, 10, 11, 3, 2]`, the answer would be `[11, 9, 9, 9, 10, 11, -1, -1, -1]` .   
**Answer:** 

In [7]:
public int[] nextLarger(int[] input) {
    int[] output = new int[input.length];
    for (int i = 0; i < input.length; i++) {
        output[i] = -1;
    }

    ArrayDeque<Integer> stack = new ArrayDeque<>();
    for (int i = input.length - 1; i >= 0; i--) {
        while (!stack.isEmpty() && input[i] >= stack.peek()) {
            stack.pop();
        }

        if (!stack.isEmpty()) {
            output[i] = stack.peek();
        }

        stack.push(input[i]);
    }

    return output;
}

System.out.println(Arrays.toString(nextLarger(new int[]{10, 7, 4, 2, 9, 10, 11, 3, 2})));

[11, 9, 9, 9, 10, 11, -1, -1, -1]


Similarly, previous larger would be

In [8]:
public int[] previousLarger(int[] input) {
    int[] output = new int[input.length];
    for (int i = 0; i < input.length; i++) {
        output[i] = -1;
    }

    ArrayDeque<Integer> stack = new ArrayDeque<>();
    for (int i = 0; i < input.length; i++) {
        while (!stack.isEmpty() && input[i] >= stack.peek()) {
            stack.pop();
        }

        if (!stack.isEmpty()) {
            output[i] = stack.peek();
        }

        stack.push(input[i]);
    }

    return output;
}

System.out.println(Arrays.toString(previousLarger(new int[]{10, 7, 4, 2, 9, 10, 11, 3, 2})));

[-1, 10, 7, 4, 10, -1, -1, 11, 3]


In similar fashion,

In [9]:
public int[] nextSmaller(int[] input) {
    int[] output = new int[input.length];
    for (int i = 0; i < input.length; i++) {
        output[i] = -1;
    }

    ArrayDeque<Integer> stack = new ArrayDeque<>();
    for (int i = input.length - 1; i >= 0; i--) {
        while (!stack.isEmpty() && input[i] <= stack.peek()) {
            stack.pop();
        }

        if (!stack.isEmpty()) {
            output[i] = stack.peek();
        }

        stack.push(input[i]);
    }

    return output;
}

System.out.println(Arrays.toString(nextSmaller(new int[]{10, 7, 4, 2, 9, 10, 11, 3, 2})));

[7, 4, 2, -1, 3, 3, 3, 2, -1]


And,

In [10]:
public int[] previousSmaller(int[] input) {
    int[] output = new int[input.length];
    for (int i = 0; i < input.length; i++) {
        output[i] = -1;
    }

    ArrayDeque<Integer> stack = new ArrayDeque<>();
    for (int i = 0; i < input.length; i++) {
        while (!stack.isEmpty() && input[i] <= stack.peek()) {
            stack.pop();
        }

        if (!stack.isEmpty()) {
            output[i] = stack.peek();
        }

        stack.push(input[i]);
    }

    return output;
}

System.out.println(Arrays.toString(previousSmaller(new int[]{10, 7, 4, 2, 9, 10, 11, 3, 2})));

[-1, -1, -1, -1, 2, 9, 10, 2, -1]


**Q 6:** Implement UNDO and REDO operations using stacks.  
**Answer:** Maintain two stack say UNDO stack and REDO stack. We use UNDO stack to keep history of all the operations that we have done. Whenever a user does UNDO operation, we simply pop the top of the element from UNDO stack and push it to the REDO stack. Then, if the user wants to do REDO operation we pop the top of the element of REDO stack and push it to UNDO. If the user performs a new operation, then we make the stack REDO empty.

**Q 7:** Insert an element at the bottom of the stack.  
**Answer:** We can solve this by maintaining an auxiliary stack.

In [11]:
public void insertAtBottom(ArrayDeque<Integer> stack, Integer value) {
    if(stack.isEmpty()) {
        stack.push(value);
        return;
    }
    
    ArrayDeque<Integer> aux = new ArrayDeque<>();
    while(!stack.isEmpty()) {
        aux.push(stack.pop());
    }
    
    stack.push(value);
    while(!aux.isEmpty()) {
        stack.push(aux.pop());
    }
}

ArrayDeque<Integer> s2 = new ArrayDeque<>();
s2.push(2);
s2.push(3);
s2.push(1);
insertAtBottom(s2, 6);
System.out.println(s2);

[1, 3, 2, 6]


Or we can use recursion.

In [12]:
public void insertAtBottomRec(ArrayDeque<Integer> stack, Integer value) {
    if(stack.isEmpty()) {
        stack.push(value);
        return;
    }
    
    Integer popped = stack.pop();
    insertAtBottomRec(stack, value);
    stack.push(popped);
}

insertAtBottom(s2, 7);
System.out.println(s2);

[1, 3, 2, 6, 7]


**Q 8:** Reverse a stack.  
**Answer:** For every stack element, insert it at the bottom. Time complexity is $O(n^2)$. Space complexity is $O(n)$

In [13]:
public void reverseInPlace(ArrayDeque<Integer> stack) {
    ArrayDeque<Integer> aux = new ArrayDeque<>();
    while(!stack.isEmpty()) {
        aux.push(stack.pop());
    }
    
    while(!aux.isEmpty()) {
        insertAtBottom(stack, aux.pop());
    }
}

reverseInPlace(s2);
System.out.println(s2);

[7, 6, 2, 3, 1]


**Q 9:** Given an array, return the number of subarrays where the leftmost element is lesser than the rightmost element. For example, if the array is `[1,4,2,5,3]`, then all possible subarrays are: `[1]`, `[4]`, `[2]`, `[5]`, `[3]`, `[1,4]`, `[2,5]`, `[1,4,2]`, `[2,5,3]`, `[4,2,5]`, `[1,4,2,5]`, `[1,4,2,5,3]`. So the count is 12.  
**Answer:** This problem can be solved iterating over each element and finding out how many elments are greater than it towards the right. This solution is $O(n^2)$.

In [14]:
public int leftMostSmaller(int[] input) {
    int count = 0;
    for(int start=0; start<input.length; start++) {
        for(int end=start; end<input.length; end++) {
            if(input[start] <= input[end]) {
                count++;
            }
        }
    }
    
    return count;
}

System.out.println(leftMostSmaller(new int[]{1,4,2,5,3}));

12


Another way to approach is to think recursively:

In [15]:
public int leftMostSmaller(int[] input, int start, int end, Set<List<Integer>> visited) {
    if(visited.contains(List.of(start, end))) return 0;
    visited.add(List.of(start, end));
    
    if(start > end) {
        return 0;
    } else if(start == end) {
        return 1;
    }
       
    int count = 0;
    if(input[start] <= input[end]) count++;
    return count + leftMostSmaller(input, start+1, end, visited) + leftMostSmaller(input, start, end-1, visited);
}

System.out.println(leftMostSmaller(new int[]{1,4,2,5,3}, 0, 4, new HashSet<>()));

12


**Q 10** Given a $N \times 2$ array, where if `A[i][0] == 1` then add `A[i][1]` to stack and return -1. If `A[i][0] == 2` then remove the most frequent item from the stack. If the most frequent item in the stack has more than one candidate, then return the one which is closest to the top.  
**Answer** To solve this we can maintain multiple stacks, each for a particular frequency

In [17]:
public int[] maxFreqStack(int[][] input) {
    int[] output = new int[input.length];
    
    Map<Integer, Integer> freqMap = new HashMap<>();  // number -> frequency
    List<ArrayDeque<Integer>> stackList = new ArrayList<>(); // all elements of stackList[i] have freq i+1
    
    for(int i=0; i<input.length; i++) {
        if(input[i][0] == 1) {
            output[i] = push(input[i][1], freqMap, stackList);
        } else if(input[i][0] == 2) {
            output[i] = pop(freqMap, stackList);
        }
    }
    
    return output;
}

private int push(int value, Map<Integer, Integer> freqMap, List<ArrayDeque<Integer>> stackList) {
    int frequency = freqMap.getOrDefault(value, 0);
    frequency++;
    freqMap.put(value, frequency);
    
    if(stackList.size() < frequency) {
        stackList.add(new ArrayDeque<Integer>());
        stackList.get(frequency-1).push(value);
    } else {
        stackList.get(frequency-1).push(value);
    }
    
    return -1;
}

private int pop(Map<Integer, Integer> freqMap, List<ArrayDeque<Integer>> stackList) {
    int i = stackList.size() - 1;
    while(i >= 0) {
        if(!stackList.get(i).isEmpty()) {
            int popped = stackList.get(i).pop();
            freqMap.put(popped, freqMap.get(popped) - 1);
            return popped;
        }
        
        i--;
    }
    
    return -1;
}

int[][] arrayInput = {{1, 5}, {1, 7}, {1, 5}, {1, 7}, {1, 4}, {1, 5}, {2, 0}, {2, 0}, {2, 0}, {2, 0}};
System.out.println(Arrays.toString(maxFreqStack(arrayInput)));

[-1, -1, -1, -1, -1, -1, 5, 7, 5, 4]


**Q 11** Given a string A representing an absolute path for a file (Unix-style). Return the string A after simplifying the absolute path. For example: `/a/./b/../../c/` is converted to `/c`  
**Answer** . We can remove the last character if it is a `/`.

In [18]:
public String simplifyDirectory(String path) {
    if(path.charAt(path.length()-1) == '/') {
        path = path.substring(0, path.length() - 1);
    }
    
    String[] parts = path.split("/");
    ArrayDeque<String> stack = new ArrayDeque<>();
    for(int i=0; i<parts.length; i++) {
        if(parts[i].equals(".") || parts[i].isEmpty()) {
            // Do nothing
        } else if(parts[i].equals("..") && !stack.isEmpty()) {
            stack.pop();
        } else {
            stack.push(parts[i]);
        }
    }
    
    StringBuilder builder = new StringBuilder();
    while (!stack.isEmpty()) {
        builder.insert(0, "/" + stack.pop());
    }
           
    return builder.toString();
}
           
String path = "/a/./b/../../c/";
System.out.println(simplifyDirectory(path));

path = "/a/b/c";
System.out.println(simplifyDirectory(path));

path = "/home/";
System.out.println(simplifyDirectory(path));

/c
/a/b/c
/home
